# Training

In [62]:
import os

DATASET_NAME = "drsprg"
DATA_BASE_DIR = f"../data/processed/{DATASET_NAME}/"
IMAGES_DIR = os.path.join(DATA_BASE_DIR, "jpgs/")
LBP_DATASET = os.path.join(DATA_BASE_DIR, "artifacts/lbp_dataset.pkl")

# Training related parameters
SEED = 42
TEST_SIZE = 0.2
RS_N_ITER = 40
CV = 20

In [63]:
import warnings

import joblib
import numpy as np
import pandas as pd
from scipy.stats import randint
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.model_selection import (
    RandomizedSearchCV,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [64]:
np.random.seed(SEED)

In [65]:
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

## LBP

In [66]:
lbp_dataset = joblib.load(LBP_DATASET)

In [67]:
X = np.array([sample[0] for sample in lbp_dataset])
y = np.array([sample[1] for sample in lbp_dataset])

In [68]:
X.shape, y.shape

((102, 1620), (102,))

In [69]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Supervised learning

#### SVM

In [70]:
def run(model, X, y, cv):
    metrics = ["precision", "recall", "f1", "accuracy"]
    results = [
        cross_val_score(model, X, y, scoring=metric, cv=cv, error_score=np.nan).mean()
        for metric in metrics
    ]
    return results

##### Radial Basis Function (RBF)

In [71]:
results = {}
rbf_svc = SVC(kernel="rbf")
results["svc_rbf"] = run(rbf_svc, X_scaled, y, CV)

##### Kernels Linear

In [72]:
linear_svc = SVC(kernel="linear")
results["svc_linear"] = run(linear_svc, X_scaled, y, CV)

#### Random Forest

In [73]:
rf = RandomForestClassifier()
results["rf"] = run(rf, X_scaled, y, CV)

#### XGBoost

In [74]:
xgbc = XGBClassifier()
results["xgbc"] = run(xgbc, X_scaled, y, CV)

### Unsupervised learning

#### K-Means

In [75]:
n_clusters = len(np.unique(y))
kmeans = KMeans(n_clusters=n_clusters, random_state=SEED)
results["kmeans"] = run(kmeans, X_scaled, y, CV)

In [76]:
columns = ["precision", "recall", "f1", "accuracy"]
pd.DataFrame(data=results, index=columns).transpose()

,precision,recall,f1,accuracy
svc_rbf,0.805833,0.883333,0.825476,0.785000
svc_linear,0.812500,0.741667,0.750238,0.743333
rf,0.796667,0.900000,0.847976,0.823333
xgbc,0.845833,0.875000,0.849048,0.821667
kmeans,0.595000,0.477500,0.507500,0.565000
